In [28]:
import pandas as pd
import json
import ast

In [29]:
df = pd.read_csv(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea_other_country\all_output_data_2.csv')

C:\Users\nguye\AppData\Local\Temp\ipykernel_33628\2396127265.py:1: DtypeWarning: Columns (1,2,3,4,7,8,9,10,13,14,15,16,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea_other_country\all_output_data_2.csv')


In [30]:
len(df)

1152758

In [31]:
org_df = df[['stt'] + [col for col in df.columns if '_org' in col]]
eng_df = df[['stt'] + [col for col in df.columns if '_eng' in col]]
kor_df = df[['stt'] + [col for col in df.columns if '_kor' in col]]

In [32]:
eng_df

,stt,기준년도_eng,참조번호_eng,결정세번_eng,품명_eng,기타품명_eng,관련 이미지_eng,시행일자_eng,물품설명_eng,결정사유_eng,...,등록일자_eng,품명선언_eng,재질성분_eng,용도_eng,분류사유_eng,세관_eng,Category_eng,회신_eng,물품설명 및 결정사유_eng,시행기관_eng
0,0,2022.0,2022_04,8471.41.90,PlanetSpark EdgeAI Box X7,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022.0,2022_02,9030.82.90,Core Development Platform (CDP),NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022.0,2022_03,8542.31.00,Qualcomm Automotive Module,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022.0,2022_01,9030.40.00,Mobile Test Platform (MTP),NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018.0,2018_20,3809.91.90,Agion® AM-HBR 4 Slurry,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152753,1156063,2002.0,NY H89839,9403.80.6040,NaN,NaN,[],2002-03-25,"H89839<br/> <br/>March 25, 2002<br/> <br/>CLA-...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,RE: The tariff classification of an outdoor di...,NaN,New York
1152754,1156064,2002.0,NY H88990,9021.10.0090; 9018.90.8000,NaN,NaN,[],2002-03-25,"H88990<br/> <br/>March 25, 2002<br/> <br/> <br...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,RE: The tariff classification of splint relate...,NaN,New York
1152755,1156065,2002.0,NY H89094,2933.99.7000,NaN,NaN,[],2002-03-25,"H89094<br/> <br/>March 25, 2002<br/> <br/> <br...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,RE: The tariff classification of Apomorphine H...,NaN,New York
1152756,1156066,2002.0,HQ 965424,"4202.92.9060, 9817.00.96",NaN,NaN,[],2002-03-25,"HQ 965424<br/> <br/>March 25, 2002<br/> <br/> ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,RE:Classification of a Carrying Case for a Blo...,NaN,Head Quarter


In [33]:
kor_df

,stt,기준년도_kor,참조번호_kor,결정세번_kor,품명_kor,기타품명_kor,관련 이미지_kor,시행일자_kor,물품설명_kor,결정사유_kor,...,등록일자_kor,품명선언_kor,재질성분_kor,용도_kor,분류사유_kor,세관_kor,Category_kor,회신_kor,물품설명 및 결정사유_kor,시행기관_kor
0,0,2022.0,2022_04,8471.41.90,PlanetSpark EdgeAI 상자 X7,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022.0,2022_02,9030.82.90,핵심 개발 플랫폼 (CDP),NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022.0,2022_03,8542.31.00,퀄컴 자동차 모듈,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022.0,2022_01,9030.40.00,모바일 시험 플랫폼 (MTP),NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018.0,2018_20,3809.91.90,Agion® AM-HBR 4 슬러리,NaN,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152753,1156063,2002.0,NY H89839,9403.80.6040,NaN,NaN,[],2002-03-25,H89839<br/><br/>2002년 3월 25일<br/><br/>CLA-2-94...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,답변 : 이탈리아로부터의 야외 식탁의 요율 분류.,NaN,New York
1152754,1156064,2002.0,NY H88990,9021.10.0090; 9018.90.8000,NaN,NaN,[],2002-03-25,H88990<br/><br/>2002년 3월 25일<br/><br/><br/>CLA...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,답변 : 멕시코로부터의 부목 관련된 항목의 요율 분류,NaN,New York
1152755,1156065,2002.0,NY H89094,2933.99.7000,NaN,NaN,[],2002-03-25,H89094<br/><br/>2002년 3월 25일<br/><br/><br/>CLA...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,"답변 : 유나이티드 킹덤으로부터, 벌크 형태에서 수입되는 염산아포모르핀 (CAS-4...",NaN,New York
1152756,1156066,2002.0,HQ 965424,"4202.92.9060, 9817.00.96",NaN,NaN,[],2002-03-25,HQ 965424<br/><br/>2002년 3월 25일<br/><br/><br/>...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Classification,답변 : 혈당 모니터링 시스템 ; 당뇨병 ; 나이로비 프로토콜을 위한 휴대용 케이스...,NaN,Head Quarter


In [34]:
def clean_data(df,language):
    df = df.rename(columns={f'기준년도_{language}': 'năm'})
    df = df.rename(columns={f'참조번호_{language}': 'số tham chiếu'})
    df = df.rename(columns={f'결정세번_{language}': 'quyết định'})
    df = df.rename(columns={f'품명_{language}': 'tên sản phẩm'})
    df = df.rename(columns={f'기타품명_{language}': 'tên sản phẩm khác'})
    df = df.rename(columns={f'관련 이미지_{language}': 'hình ảnh'})
    df = df.rename(columns={f'시행일자_{language}': 'ngày hiệu lực'})
    df = df.rename(columns={f'물품설명_{language}': 'mô tả sản phẩm'})
    df = df.rename(columns={f'결정사유_{language}': 'lý do quyết định'})
    df = df.rename(columns={f'키워드_{language}': 'từ khóa'})
    df = df.rename(columns={f'다른이름_{language}': 'tên khác'})
    df = df.rename(columns={f'이슈장소_{language}': 'nơi phát hành'})
    df = df.rename(columns={f'등록일자_{language}': 'ngày đăng ký'})
    df = df.rename(columns={f'품명선언_{language}': 'tên sản phẩm khai báo'})
    df = df.rename(columns={f'재질성분_{language}': 'thành phần vật liệu'})
    df = df.rename(columns={f'용도_{language}': 'sử dụng'})
    df = df.rename(columns={f'분류사유_{language}': 'lý do phân loại'})
    df = df.rename(columns={f'세관_{language}': 'hải quan'})
    df = df.rename(columns={f'Category_{language}': 'Category'})
    df = df.rename(columns={f'회신_{language}': 'hồi đáp'})
    df = df.rename(columns={f'물품설명 및 결정사유_{language}': 'Mô tả sản phẩm và lý do quyết định'})
    df = df.rename(columns={f'시행기관_{language}': 'Cơ quan thực hiện'})
    
    # Replace missing values with "" in column: 'ngày hiệu lực'
    df = df.fillna({'ngày hiệu lực': df['năm']})
    df = df.rename(columns={'ngày hiệu lực': 'doc_date'})
    
    # Rename column 'số tham chiếu' to 'doc_no'
    df = df.rename(columns={'số tham chiếu': 'doc_no'})
    df = df[df['doc_no'].notna()]
    
    # Rename column 'tên sản phẩm' to 'product_name'
    df = df.rename(columns={'tên sản phẩm': 'product_name'})
    
    # Rename column 'quyết định' to 'hs'
    df = df.rename(columns={'quyết định': 'hs'})
    
    # Derive column 'implementing_organization' from columns: 'hải quan', 'Cơ quan thực hiện'
    df.insert(
        23,  
        'implementing_organization',
        df.apply(
            lambda row: row['Cơ quan thực hiện'] 
            if pd.notnull(row['Cơ quan thực hiện']) and row['Cơ quan thực hiện'] != '' 
            else (row['hải quan'] if pd.notnull(row['hải quan']) and row['hải quan'] != '' else ''),
            axis=1
        )
    )
    
    # Derive column 'product_properties' from columns: mô tả sản phẩm, 'thành phần vật liệu'
    df.insert(
        24, 
        'product_properties',
        df.apply(
            lambda row: row['mô tả sản phẩm'] 
            if pd.notnull(row['mô tả sản phẩm']) and row['mô tả sản phẩm'] != '' 
            else (row['thành phần vật liệu'] if pd.notnull(row['thành phần vật liệu']) and row['thành phần vật liệu'] != '' else ''),
            axis=1
        )
    )
    
    # Derive column 'decision_reason' from columns: 'lý do phân loại', 'Mô tả sản phẩm và lý do quyết định'
    df.insert(
        25,  # Vị trí cột
        'decision_reason',
        df.apply(
            lambda row: row['lý do phân loại'] 
            if pd.notnull(row['lý do phân loại']) and row['lý do phân loại'] != '' 
            else (row['Mô tả sản phẩm và lý do quyết định'] if pd.notnull(row['Mô tả sản phẩm và lý do quyết định']) and row['Mô tả sản phẩm và lý do quyết định'] != '' else ''),
            axis=1
        )
    )
    
    # Replace all instances of "/clip/framework/filedownload/" with "https://unipass.customs.go.kr/clip/framework/filedownload/" in column: 'hình ảnh'
    df.insert(
        26,  # Vị trí cột
        'original_link',
        df['hình ảnh'].str.replace(
            "/clip/framework/filedownload/", 
            "https://unipass.customs.go.kr/clip/framework/filedownload/", 
            case=False, 
            regex=False
        )
    )
    df.fillna('')
    return df

In [35]:
def toRealList(strList):
    if pd.isna(strList) or strList == '':
        return
    lst = ast.literal_eval(strList)
    return json.loads(json.dumps(lst))
def tolist(df,col):
    df[col] = df[col].apply(lambda x: str(x).split("<br/>"))

In [36]:
def make_json_temp(df):
    df = df.applymap(lambda x: x.strip("<br/>") if isinstance(x, str) else x)
    df['id'] = df.reset_index().index + 1
    id = df.pop("id")
    df.insert(0, id.name, id)
    df = df.map(lambda x: x.replace('\n', '<br/>').replace('\xa0','') if isinstance(x, str) else x)
    df["hs"] = df["hs"].str.replace(r"(<br/>\s*)+", " ", regex=True)
    df["product_properties"] = df["product_properties"].str.replace(r"(<br/>\s*)+", "<br/> ", regex=True)
    df["decision_reason"] = df["decision_reason"].str.replace(r"(<br/>\s*)+", "<br/> ", regex=True)
    df['img_links'] = df['img_links'].apply(lambda x: toRealList(x))
    # print(df)
    for col in ['product_name','product_properties','decision_reason']:
        tolist(df,col)
    return df


In [37]:
all_link_df = pd.read_csv(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea_other_country\all_link_list.csv')

In [38]:
nation_name_df = pd.read_excel(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_korea_other_country\nation_name.xlsx')

In [39]:
all_link_df_with_nation_name_df = pd.merge(all_link_df,nation_name_df,how='left',left_on='cntyCd',right_on='code')

In [40]:
org_df_clean = clean_data(org_df.copy(),'org')
org_df_clean = pd.merge(org_df_clean,all_link_df_with_nation_name_df,how='left',on='stt')
org_df_final = org_df_clean[['doc_no','doc_date','implementing_organization','hs','product_name','product_properties','decision_reason','original_link','Eng_nation_name']].rename(columns={'Eng_nation_name': 'nation','original_link': 'img_links'}).fillna('')
org_df_final.head(5)

,doc_no,doc_date,implementing_organization,hs,product_name,product_properties,decision_reason,img_links,nation
0,2022_04,2022.0,,8471.41.90,PlanetSpark EdgeAI Box X7,,,[],Singapore
1,2022_02,2022.0,,9030.82.90,Core Development Platform (CDP),,,[],Singapore
2,2022_03,2022.0,,8542.31.00,Qualcomm Automotive Module,,,[],Singapore
3,2022_01,2022.0,,9030.40.00,Mobile Test Platform (MTP),,,[],Singapore
4,2018_20,2018.0,,3809.91.90,Agion® AM-HBR 4 Slurry,,,[],Singapore


In [41]:
eng_df_clean = clean_data(eng_df.copy(),'eng')
eng_df_clean = pd.merge(eng_df_clean,all_link_df_with_nation_name_df,how='left',on='stt')
eng_df_final = eng_df_clean[['doc_no','doc_date','implementing_organization','hs','product_name','product_properties','decision_reason','original_link','Eng_nation_name']].rename(columns={'Eng_nation_name': 'nation','original_link': 'img_links'}).fillna('')
eng_df_final.head(5)

,doc_no,doc_date,implementing_organization,hs,product_name,product_properties,decision_reason,img_links,nation
0,2022_04,2022.0,,8471.41.90,PlanetSpark EdgeAI Box X7,,,[],Singapore
1,2022_02,2022.0,,9030.82.90,Core Development Platform (CDP),,,[],Singapore
2,2022_03,2022.0,,8542.31.00,Qualcomm Automotive Module,,,[],Singapore
3,2022_01,2022.0,,9030.40.00,Mobile Test Platform (MTP),,,[],Singapore
4,2018_20,2018.0,,3809.91.90,Agion® AM-HBR 4 Slurry,,,[],Singapore


In [42]:
kor_df_clean = clean_data(kor_df.copy(),'kor')
kor_df_clean = pd.merge(kor_df_clean,all_link_df_with_nation_name_df,how='left',on='stt')
kor_df_final = kor_df_clean[['doc_no','doc_date','implementing_organization','hs','product_name','product_properties','decision_reason','original_link','Kr_nation_name']].rename(columns={'Kr_nation_name': 'nation','original_link': 'img_links'}).fillna('')
kor_df_final.head(5)

,doc_no,doc_date,implementing_organization,hs,product_name,product_properties,decision_reason,img_links,nation
0,2022_04,2022.0,,8471.41.90,PlanetSpark EdgeAI 상자 X7,,,[],싱가포르
1,2022_02,2022.0,,9030.82.90,핵심 개발 플랫폼 (CDP),,,[],싱가포르
2,2022_03,2022.0,,8542.31.00,퀄컴 자동차 모듈,,,[],싱가포르
3,2022_01,2022.0,,9030.40.00,모바일 시험 플랫폼 (MTP),,,[],싱가포르
4,2018_20,2018.0,,3809.91.90,Agion® AM-HBR 4 슬러리,,,[],싱가포르


In [43]:
old_kor_data = pd.read_excel("ko_prod_class_22-10_with_imglink.xlsx",
                    sheet_name="Sheet1",
                    skiprows=2).fillna("").iloc[:,1:10]

In [44]:
kor_data = pd.concat([kor_df_final,old_kor_data],ignore_index=True)

In [45]:
kor_data_json_final  = make_json_temp(kor_data)

C:\Users\nguye\AppData\Local\Temp\ipykernel_33628\749152255.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip("<br/>") if isinstance(x, str) else x)


In [54]:
with open('ko_prod_class_06_01.json', 'w') as f:
    json.dump(json.loads(kor_data_json_final.to_json(orient="records")), f)

In [47]:
old_eng_data = pd.read_excel("en_prod_class.xlsx",
                    sheet_name="Sheet1",
                    skiprows=2).fillna("").iloc[:,1:10]

In [48]:
eng_data = pd.concat([eng_df_final,old_eng_data],ignore_index=True)

In [49]:
eng_data_json_final  = make_json_temp(eng_data)

C:\Users\nguye\AppData\Local\Temp\ipykernel_33628\749152255.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip("<br/>") if isinstance(x, str) else x)


In [50]:
len(eng_data_json_final)

1204993

In [51]:
eng_data_json_final

,id,doc_no,doc_date,implementing_organization,hs,product_name,product_properties,decision_reason,img_links,nation
0,1,2022_04,2022.0,,8471.41.90,[PlanetSpark EdgeAI Box X7],[],[],[],Singapore
1,2,2022_02,2022.0,,9030.82.90,[Core Development Platform (CDP)],[],[],[],Singapore
2,3,2022_03,2022.0,,8542.31.00,[Qualcomm Automotive Module],[],[],[],Singapore
3,4,2022_01,2022.0,,9030.40.00,[Mobile Test Platform (MTP)],[],[],[],Singapore
4,5,2018_20,2018.0,,3809.91.90,[Agion® AM-HBR 4 Slurry],[],[],[],Singapore
...,...,...,...,...,...,...,...,...,...,...
1204988,1204989,Item Classification 3-2213,2015-08-13,Customs Valuation and Classification Institute,Item ①: 8525.80-1090_x000D_ Item ②: 8525.80-30...,"[① Parrot AR ; PF721900 SAND ; CHINA, ② Parro...","[Overview, ㆍ A drone with a body and camera e...",[The goods in question are goods that implemen...,[https://unipass.customs.go.kr/clip/framework/...,Korea
1204989,1204990,Item Classification 3-6155,2024-11-18,Customs Valuation and Classification Institute,9032.20-0000,"[AUTO PRESSURE REGULATOR , ① UR-Z724, ② GR-314F]","[ㅇ Product Overview , - A product consisting ...",[ㅇ In Note 7 to Chapter 90 of the Tariff Sched...,[https://unipass.customs.go.kr/clip/framework/...,Korea
1204990,1204991,Item Classification 3-6059,2017-11-09,Customs Valuation and Classification Institute,8536.90-1000,"[① Junction boxes ; XJPV-601A ; CHINA, ② Junct...","[ㅇ Overview, - A product that combines a junc...",[ㅇ Insulated wires (Tariff No. 8544) can be cl...,[https://unipass.customs.go.kr/clip/framework/...,Korea
1204991,1204992,Review Policy and-100040,2006-08-01,Customs Service,8538.90-9000,[IC ; TLE4208G ;; Made in Germany],"[ㅇ Composition and shape, - A composite chip-...","[ㅇ Tariff No. 8538 classifies ""parts used sole...",[],Korea


In [53]:
with open('eng_prod_class_06_01.json', 'w') as f:
    json.dump(json.loads(eng_data_json_final.to_json(orient="records")), f)